In [ ]:
import torch
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, classification_report, precision_score

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Master/active_learning2.csv')
df['textOriginal'] = df['textOriginal'].astype(str)
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x > 0 else 0 if x <= 0 else x)
df['title'] = df['title'].astype(str)
df['video_or_channel'] = df['video_or_channel'].astype(int)
df = df.dropna(subset='sentiment')
df['comment'] = "[CHANNEL]" +df['channel_name'] + "[COMMENT] " + df['textOriginal'] + " [TITLE] " + df['title']